In [3]:
import os
import sys
import datetime
import math
import pandas as pd
import numpy as np
import quant_utils.data_moudle as dm
from quant_utils.db_conn import DB_CONN_JJTG_DATA
from quant_utils.performance import Performance, periods_performance

In [4]:
def get_simulated_portfolio_nav(start_date, end_date):
    query_sql = f"""
WITH t AS (
	WITH a AS (
		SELECT
			TRADE_DT,
			TICKER_SYMBOL,
			S_DQ_PCTCHANGE 
		FROM
			fund_index_eod 
		WHERE
			TICKER_SYMBOL IN ( '930950', '885001', '885062', '885008', 'H11025', '930609' ) UNION
		SELECT
			TRADE_DT,
			TICKER_SYMBOL,
			S_DQ_PCTCHANGE 
		FROM
			aindex_eod_prices 
		WHERE
			1 = 1 
		AND TICKER_SYMBOL IN ( '000300', '000905', '000852' )) SELECT
		a.TRADE_DT,
		sum( CASE TICKER_SYMBOL WHEN "930950" THEN S_DQ_PCTCHANGE / 100 ELSE 0 END ) AS "930950",
		sum( CASE TICKER_SYMBOL WHEN "885001" THEN S_DQ_PCTCHANGE / 100 ELSE 0 END ) AS "885001",
		sum( CASE TICKER_SYMBOL WHEN "885062" THEN S_DQ_PCTCHANGE / 100 ELSE 0 END ) AS "885062",
		sum( CASE TICKER_SYMBOL WHEN "885008" THEN S_DQ_PCTCHANGE / 100 ELSE 0 END ) AS "885008",
		sum( CASE TICKER_SYMBOL WHEN "H11025" THEN S_DQ_PCTCHANGE / 100 ELSE 0 END ) AS "H11025",
		sum( CASE TICKER_SYMBOL WHEN "930609" THEN S_DQ_PCTCHANGE / 100 ELSE 0 END ) AS "930609",
		sum( CASE TICKER_SYMBOL WHEN "000300" THEN S_DQ_PCTCHANGE / 100 ELSE 0 END ) AS "000300",
		sum( CASE TICKER_SYMBOL WHEN "000905" THEN S_DQ_PCTCHANGE / 100 ELSE 0 END ) AS "000905",
		sum( CASE TICKER_SYMBOL WHEN "000852" THEN S_DQ_PCTCHANGE / 100 ELSE 0 END ) AS "000852" 
	FROM
		a 
	GROUP BY
		TRADE_DT 
	ORDER BY
		TRADE_DT 
	) SELECT
	t.TRADE_DT,
	round(
		exp(
			sum(
			log( t.`H11025` * 0.75 + t.`885062` * 0.25 + 1 )) over ( ORDER BY trade_dt )),
		8 
	) AS '知己优选-货币小福星',
	round( exp( sum( log( t.`885062` * 1 + 1 )) over ( ORDER BY trade_dt )), 8 ) AS '知己优选-短债增强',
	round(
		exp(
			sum(
			log( t.`930950` * 0.1 + t.`930609` * 0.9 + 1 )) over ( ORDER BY trade_dt )),
		8 
	) AS '知己优选-月月享',
	round(
		exp(
			sum(
			log( t.`930950` * 0.2 + t.`930609` * 0.8 + 1 )) over ( ORDER BY trade_dt )),
		8 
	) AS '知己优选-固收佳进取',
	round(
		exp(
			sum(
			log( t.`930950` * 0.5 + t.`930609` * 0.5 + 1 )) over ( ORDER BY trade_dt )),
		8 
	) AS '知己优选-均衡小确幸',
	round(
		exp(
			sum(
			log( t.`930950` * 0.95 + t.`H11025` * 0.05 + 1 )) over ( ORDER BY trade_dt )),
		8 
	) AS '知己优选-进取全明星',
	round(
		exp(
			sum(
			log( t.`000300` * 0.95 + t.`H11025` * 0.05 + 1 )) over ( ORDER BY trade_dt )),
		8 
	) AS '知己优选-300增强',
	round(
		exp(
			sum(
			log( t.`000905` * 0.95 + t.`H11025` * 0.05 + 1 )) over ( ORDER BY trade_dt )),
		8 
	) AS '知己优选-500增强',
	round(
		exp(
			sum(
			log( t.`000852` * 0.95 + t.`H11025` * 0.05 + 1 )) over ( ORDER BY trade_dt )),
	8 
	) AS '知己优选-1000增强'
	
	FROM
        t 
    WHERE
        1 = 1 
        AND TRADE_DT BETWEEN '{start_date}' 
        AND '{end_date}'
    ORDER BY TRADE_DT
    """
    df = DB_CONN_JJTG_DATA.exec_query(query_sql)
    df = df.set_index("TRADE_DT")
    return df

In [5]:
# perf_result = []
# win_rate_result = []
# df = get_simulated_portfolio_nav("20140101", '20240430')
# df = periods_performance(df["知己优选-短债优选"])
# df.to_excel("C:/Users/Wilcoxon/Desktop/短债.xlsx")

In [6]:
perf_result = []
win_rate_result = []
period_result = []
df = get_simulated_portfolio_nav("20190101", '20240731')


for col in df.columns:
    portfolio_perf = Performance(df[col])
    perf_df = portfolio_perf.stats().T
    period_perf = periods_performance(df[col])
    period_perf["组合"] = col
    perf_df.index = [col]
    win_rate = portfolio_perf.rolling_ret_stats().reset_index().rename(columns={"index": "指标"})
    win_rate["组合"] = col
    perf_result.append(perf_df)
    win_rate_result.append(win_rate)
    period_result.append(period_perf)
perf_result = pd.concat(perf_result)
win_rate_result = pd.concat(win_rate_result)
win_rate_result = win_rate_result.query("指标 == '收益率中位数' or 指标 == '胜率'")
period_result = pd.concat(period_result)

with pd.ExcelWriter("d:/组合风险特征数据更新.xlsx") as w:
    perf_result.to_excel(w, sheet_name='风险特征')
    win_rate_result.to_excel(w, sheet_name='持有时间和胜率')
    period_result.to_excel(w, sheet_name='区间表现')
# df.to_excel("d:/方案.xlsx", sheet_name='组合净值')

D:\FundResearch\quant_utils\performance.py:601: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return pd.DataFrame.from_dict(perf_dict).applymap(lambda x: format(x, ".2%"))
D:\FundResearch\quant_utils\performance.py:601: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return pd.DataFrame.from_dict(perf_dict).applymap(lambda x: format(x, ".2%"))
D:\FundResearch\quant_utils\performance.py:601: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return pd.DataFrame.from_dict(perf_dict).applymap(lambda x: format(x, ".2%"))
D:\FundResearch\quant_utils\performance.py:601: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return pd.DataFrame.from_dict(perf_dict).applymap(lambda x: format(x, ".2%"))
D:\FundResearch\quant_utils\performance.py:601: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return pd.DataFrame.from_dic